In [58]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()
with open("../login.json", "r") as j:
    misDatos = json.load(j)
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [37]:
nuevos = "C:/Users/jmartinez/Documents/final_metlife_base.xlsx"
Pacientes_Archivo2 = pd.read_excel(nuevos)
Pacientes_Archivo = pd.DataFrame()
Pacientes_Archivo.head(5)



""


In [38]:
Pacientes_Archivo = Pacientes_Archivo2

In [39]:
df3 = pd.DataFrame()
Pacientes_Archivo.sort_values("Fecha_chilena", inplace=True)
Pacientes_Archivo.RUT_T.fillna(Pacientes_Archivo["Rut"], inplace=True)

Pacientes_Archivo_moviles = Pacientes_Archivo[(Pacientes_Archivo["Fecha"] >= "2023-02-01") & (Pacientes_Archivo["Fecha"] <= "2024-01-31")]
Pacientes_Archivo_moviles["llave"] = "Enero" + Pacientes_Archivo["Rut"].astype(str)

Pacientes_Archivo.head(5)
Pacientes_Archivo["llave"] = Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Rut"].astype(str)
#Pacientes_Archivo["llaveEnero"] = "Enero"+ Pacientes_Archivo["Rut_Titular_y"].astype(str)

Pacientes_Archivo["Familia_Mensual"] = Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Mes"].astype(str) + Pacientes_Archivo["RUT_T"].astype(str)
Pacientes_Archivo["rut_Unico_Mensual"] =  Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Mes"].astype(str) + Pacientes_Archivo["Rut"].astype(str)
Pacientes_Archivo["rut_Unico_Anual"] =  Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Rut"].astype(str)
Pacientes_Archivo["uso_titulares"] = np.where(Pacientes_Archivo["RUT_T"] == Pacientes_Archivo["Rut"], 1, 0)
Pacientes_Archivo["uso_titulares_Mes"] = np.where(Pacientes_Archivo["RUT_T"] == Pacientes_Archivo["Rut"], Pacientes_Archivo["Rut"].astype(str) + Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Mes"].astype(str) , "NO")
Pacientes_Archivo["uso_titulares_Año"] = np.where(Pacientes_Archivo["RUT_T"] == Pacientes_Archivo["Rut"], Pacientes_Archivo["Rut"].astype(str) + Pacientes_Archivo["Año"].astype(str) , 0)
#Pacientes_Archivo["hora_Agenda"] = Pacientes_Archivo.Fecha_chilena.hour
#.astype(str)

Pacientes_Archivo["uso_titulares_Mes"] = Pacientes_Archivo["uso_titulares_Mes"].astype(str)


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_13580\817814855.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pacientes_Archivo_moviles["llave"] = "Enero" + Pacientes_Archivo["Rut"].astype(str)


In [40]:
df3 = Pacientes_Archivo_moviles[[ "Index","llave"]]
df3 = df3.drop_duplicates(
    [
        "llave",
    ],
    keep="first",
)
df3["año_movil"] = 1 
df3["año_movil"] = 1 
df3 = df3[[ "Index","año_movil"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.año_movil.fillna(0, inplace=True)


In [41]:
#Pacientes_Archivo.head()

In [42]:
df3 = Pacientes_Archivo[[ "Index","llave"]]
df3 = df3.drop_duplicates(
    [
        "llave",
    ],
    keep="first",
)
df3["año_movil_CalenDario"] = 1 
df3 = df3[[ "Index","año_movil_CalenDario"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.año_movil_CalenDario.fillna(0, inplace=True)


In [43]:
df3 = Pacientes_Archivo[[ "Index","Familia_Mensual"]]
df3 = df3.drop_duplicates(
    [
        "Familia_Mensual",
    ],
    keep="first",
)
df3["Familia_unica_Mensual_CT"] = 1 
df3 = df3[[ "Index","Familia_unica_Mensual_CT"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.Familia_unica_Mensual_CT.fillna(0, inplace=True)
df3.head()

,Index,Familia_unica_Mensual_CT
0,11,1
1,10,1
2,3,1
3,4,1
4,12,1


In [44]:
#["rut Unico Mensual"]
df3 = Pacientes_Archivo[[ "Index","rut_Unico_Mensual"]]
df3 = df3.drop_duplicates(
    [
        "rut_Unico_Mensual",
    ],
    keep="first",
)
df3["rut_Unico_Mensual_CT"] = 1 
df3 = df3[[ "Index","rut_Unico_Mensual_CT"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.rut_Unico_Mensual_CT.fillna(0, inplace=True)
df3.head()

,Index,rut_Unico_Mensual_CT
0,11,1
1,10,1
2,3,1
3,4,1
4,12,1


In [45]:
df3 = Pacientes_Archivo[["Index","uso_titulares_Mes"]]
df3 = df3.drop_duplicates(
    [
        "uso_titulares_Mes",
    ],
    keep="first",
)
df3["uso_titulares_Mensual_CT"] = 1 
df3 = df3[[ "Index","uso_titulares_Mensual_CT"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.uso_titulares_Mensual_CT.fillna(0, inplace=True)
df3.head()

,Index,uso_titulares_Mensual_CT
0,11,1
1,10,1
2,3,1
3,4,1
4,12,1


In [46]:
df3 = Pacientes_Archivo[["Index","uso_titulares_Año"]]
df3 = df3.drop_duplicates(
    [
        "uso_titulares_Año",
    ],
    keep="first",
)
df3["uso_titulares_unicoanual_CT"] = 1 
df3 = df3[[ "Index","uso_titulares_unicoanual_CT"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.uso_titulares_unicoanual_CT.fillna(0, inplace=True)
df3.head()

,Index,uso_titulares_unicoanual_CT
0,11,1
1,10,1
2,3,1
3,4,1
4,12,1


In [47]:
derty =Pacientes_Archivo.groupby("Rut").count()
derty = derty[["Nombre"]]
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=derty,
    how="left",
    left_on="Rut",
    right_on="Rut",
)

In [48]:
Pacientes_Archivo = Pacientes_Archivo.rename(
    columns={
        "Nombre_x": "Nombre",
        "Nombre_y": "Repeticiones",
        
    }
)

In [49]:
#derty.head(100)

In [50]:


#derty.head(100)

In [51]:
Pacientes_Archivo.head()

,Codigo Ficha,Rut,Nombre,Sexo,Campaña,Holding,Empleador,Poliza,Fecha,Fecha_chilena,...,uso_titulares,uso_titulares_Mes,uso_titulares_Año,año_movil,año_movil_CalenDario,Familia_unica_Mensual_CT,rut_Unico_Mensual_CT,uso_titulares_Mensual_CT,uso_titulares_unicoanual_CT,Repeticiones
0,602d26fd8782bd731,156596981,Cifuentes Toloza Abraham Fernando,Masculino,MetLife,NaN,Multibien Servicio Bienestar Multibien - Holdi...,340018776,2022-01-02 20:58:36,2022-01-02,...,1,15659698120221,1565969812022,0.0,1.0,1.0,1.0,1.0,1.0,10
1,5f52617a298d149b2,129847514,Norambuena Alvaro Edson Benavides,Masculino,MetLife,NaN,Ingrammicro,340017722,2022-01-02 20:51:50,2022-01-02,...,1,12984751420221,1298475142022,0.0,1.0,1.0,1.0,1.0,1.0,1
2,5fff231864cbd82d4,154611428,Zamora Camilo Allan Latorre,Masculino,MetLife,NaN,clalemana,340006130,2022-01-02 14:26:40,2022-01-02,...,1,15461142820221,1546114282022,0.0,1.0,1.0,1.0,1.0,1.0,48
3,606f6f10644fd3e9a,196356959,Undurraga Camila Haritcalde,Femenino,MetLife,NaN,Corredora de Seguros Security Ltda,340003421,2022-01-02 14:39:26,2022-01-02,...,0,NO,0,0.0,1.0,1.0,1.0,1.0,1.0,3
4,5eabd492acb748fb0,120823167,"ECHAGÜE, VERONICA PATRICIA VALENZUELA",Femenino,MetLife,NaN,Metlife,340018133,2022-01-02 21:08:45,2022-01-02,...,1,12082316720221,1208231672022,0.0,1.0,1.0,1.0,1.0,1.0,9


In [52]:
#Pacientes_Archivo.head(5)


In [53]:
#from pivottablejs import pivot_ui

In [55]:
dertyuiop = Pacientes_Archivo.to_excel("C:/Users/jmartinez/Documents/reportes_2111.xlsx", index=False)
#brand_Group_DF  = sf.smart_query(dertyuiop)
brand_Group_DF = brand_Group_DF[["AppointmentNumber","CreatedDate"]]


In [64]:
dertyuiop = "SELECT Id, SchedStartTime, AppointmentNumber,CreatedDate from ServiceAppointment"
brand_Group_DF  = sf.smart_query(dertyuiop)
#dertyuiop
brand_Group_DF = brand_Group_DF[["AppointmentNumber","CreatedDate"]]
#Pacientes_Archivo.to_excel("C:/Users/jmartinez/Documents/reportes_2111.xlsx", index=False)
#pivot_ui(Pacientes_Archivo)
brand_Group_DF2 = pd.DataFrame()
brand_Group_DF2["AppointmentNumber"] = "F" + brand_Group_DF["AppointmentNumber"]
brand_Group_DF2["CreatedDate"] = brand_Group_DF["CreatedDate"]
brand_Group_DF2.head()
#CreatedDate
#brand_Group_DF.head()


100%|██████████| 1/1 [00:12<00:00, 12.01s/it]


,AppointmentNumber,CreatedDate
0,FSA-0030,1668868174000
1,FSA-0032,1668870893000
2,FSA-0033,1668871031000
3,FSA-0045,1668873317000
4,FSA-0059,1669045284000
